# Prime Python API Overview

## Outline

1. Configuring credentials
2. Retrieving objects
   - Loss Sets
   - Layers
   - Portfolios
   - Event Catalogs
   - Analysis Profiles
3. Uploading data
   - Loss Sets
4. Creating new objects
   - Layers
   - Portfolios
5. Modifying objects
   - Modify a layer
   - Modify a portfolio
6. Creating views
   - Layer Views
   - Portfolio Views
7. Querying metrics
   - Layer metrics
   - Portfolio metrics
8. Performance optimization
9. Advanced metrics
   - Co-Metrics
   - Window Metrics

## Configuring Credentials

In [ ]:
import analyzere
from getpass import getpass

In [ ]:
analyzere.base_url = "[Enter your server name here]"

In [ ]:
analyzere.username = "[Enter your user name here]"

In [ ]:
analyzere.password = getpass()

## Retrieving objects

### Loss Sets

In [ ]:
from analyzere import LossSet

#### Listing loss sets

In [ ]:
ls_from_list = LossSet.list()

In [ ]:
ls_from_list[0]

#### Retrive a single loss set

In [ ]:
ls_from_id = LossSet.retrieve(ls_from_list[0].id)

In [ ]:
ls_from_id

#### Search loss sets

In [ ]:
ls_from_search = LossSet.list(search="[Enter loss set search term]")

In [ ]:
[ (ls.id, ls.description) for ls in ls_from_search]

### Layers

In [ ]:
from analyzere import Layer

#### Listing layers

In [ ]:
l_from_list = Layer.list()

In [ ]:
[ (l.id, l.description) for l in l_from_list[:10]]

#### Retrieve a single layer

In [ ]:
l_from_id = Layer.retrieve(l_from_list[0].id)

In [ ]:
l_from_id

#### Search layers

In [ ]:
l_from_search = Layer.list(search="[Enter search term]")

In [ ]:
[ (l.id, l.description, l.meta_data["[Enter meta data field]"]) for l in l_from_search]

#### Search by metadata fields

##### Search by Underwriter

In [ ]:
l_from_metadata = Layer.list(metaquery='meta_data["underwriter"]="Susan Murphy"')

In [ ]:
l_from_metadata.meta.total_count

##### Search by Program ID

In [ ]:
l_from_program_metadata = Layer.list(metaquery='meta_data["program_id"]="1"')

In [ ]:
l_from_program_metadata.meta.total_count

##### Limit the fields returned

In [ ]:
l_limited_fields = Layer.list(fields="id,description")

In [ ]:
l_limited_fields[0]

##### Limit the number of results returned

In [ ]:
l_limited_responses = Layer.list(limit=10)

In [ ]:
len(l_limited_responses)

In [ ]:
l_limited_responses.meta.total_count

##### Skip a number of results returned

In [ ]:
l_offset = Layer.list(offset=10, limit=10)

In [ ]:
len(l_offset)

In [ ]:
l_offset.meta.total_count

##### Sort responses

Order responses by descending creation date/time.

In [ ]:
l_created_desc = Layer.list(ordering="-created")

Order responses by ascending modification date/time.

In [ ]:
l_modified_asc = Layer.list(ordering="modified")

### Portfolios

In [ ]:
from analyzere import Portfolio

#### List portfolios

In [ ]:
portfolios = Portfolio.list()

In [ ]:
portfolios.meta.total_count

#### Retrieve single portfolio

In [ ]:
portfolio = Portfolio.retrieve(portfolios[0].id)

In [ ]:
portfolio

### Event Catalog

In [ ]:
from analyzere import EventCatalog

#### List event catalogs

In [ ]:
catalogs = EventCatalog.list()

In [ ]:
catalogs.meta.total_count

#### Retrieve single event catalog

In [ ]:
catalog = EventCatalog.retrieve(catalogs[0].id)

In [ ]:
catalog

### Analysis Profile

In [ ]:
from analyzere import AnalysisProfile

#### List analysis profiles

In [ ]:
analysis_profiles = AnalysisProfile.list()

In [ ]:
analysis_profiles.meta.total_count

### Automatic Dereferencing

As you can see above, the Prime API makes ample use of cross-referencing objects that alreay exist on the server. The Python-bindings can automatically dereference those references and "lazy-load" objects from the server when they are being access.

For example, we will retrieve a Layer and then implicitly dereference and retrieve the associated Loss Set object and, in turn, the Loss Set's Event Catalog object. Note, the retrieval of those objects only includes their header data and not any underlying large datasets such as loss set data or event catalog data.

Also note, the dereferencing occurs implicitly on a per-access basis, which is not always the most efficient approach as it is inherently sequential and unknowingly to the user/programmer may incur unnecessary requests.

In [ ]:
Layer.list(search="[Enter search term]", limit=1)[0].loss_sets[0].event_catalogs[0].description

## Uploading data

Create a new loss set object and upload data associated with that newly created loss set object. However, first we need an event catalog as each loss set must be associated with an event catalog.

In [ ]:
ec = EventCatalog.retrieve('[Enter a GUID here]')
print("ec 1 : " + str(ec.id))
# OR search for an event catalog by name (searches always return lists)
ec = EventCatalog.list(search="[Enter the name for an event catalog here]", limit=1)[0]
print("ec 2 : " + str(ec.id))

Now we can create the loss set object.

In [ ]:
ls = LossSet(
    # We always must specify the type of loss set. In this case we will upload a YELT loss set (Trial/Year, Time/Sequence, EventId, Loss)
    type="YELTLossSet",
    # We also must specify an event catalog. We use the one we retrieved above.
    event_catalogs=[ec],
    # A description is required and always good to be meaningfully selected with respect to the client-application.
    description="Test Loss Set for Python Training",
    # We also must specify the currency in which losses in the loss set are stored.
    currency="USD",
    # For YELTLossSets we must set a start date of where the simulation begins. You can also use `datetime` here.
    start_date="2022-01-01T00:00:00Z",
    # For YELTLossSets we must specify how many trials / years are represented by the loss set.
    trial_count=10000,
)    

This only created an object client-side. Now we need to save it to the server.

In [ ]:
ls.save()

Status `ready` means that the object is ready to receive data.

In [ ]:
ls.status

Now we upload the data in CSV format. `upload_data` blocks until the status is set to "Processing Successful" or "Processing Failed".

In [ ]:
ls.upload_data(open("local.csv", "rb"))

The loss set is now ready to be used.

## Creating Objects

### Create a Layer

Layers are all created in a similar manner: A Python `Layer` object is created with arguments that match the required JSON attributes. Here we will create a CatXL layer and use the loss set uploaded above as the subject losses to this layer. However, we must first import additional types from the library.

In [ ]:
from analyzere import MonetaryUnit, Reinstatement

In [ ]:
layer = Layer(
    # Type of layer: CatXL
    type="CatXL",
    # Loss sets associated with the layer
    loss_sets=[ls],
    # 3M USD attachment
    attachment=MonetaryUnit(3e6, "USD"),
    # 10M USD limit
    limit=MonetaryUnit(10e6, "USD"),
    # Franchise deductible is a required field, but we set it to 0 here.
    franchise=MonetaryUnit(0, "USD"),
    # The reinsurer's participation: 25%
    participation=0.25,
    # Inception date (optional, inclusive, supports `datetime`)
    inception_date="2022-01-01T00:00:00Z",
    # Expiry date (optional, exclusive, supports `datetime`)
    expiry_date="2023-01-01T00:00:00Z",
    # Description (optional)
    description="Test Layer for Python Training",
    # Additional metadata fields (optional)
    meta_data={
        "Test-LayerId": 12345,
        "Test-Purpose": "Training",
        "Test-Version": 1,
        "Test-Latest": True,
    }
).save()

In [ ]:
layer

### Create a Portfolio

In [ ]:
from analyzere import Portfolio

A portfolio is simply a collection of layers. In this example, we will create a portfolio of only one layer.

In [ ]:
portfolio = Portfolio(
    # A list of layers that are to me included in the portfolio. This can also be done
    layers=[layer],
    name="Test Portfolio for Python Training",
    description="Test Portfolio for Python Training",
    meta_data={
        "Test-PortfolioId": 12345,
        "Test-Purpose": "Training",
        "Test-Version": 1,
        "Test-Latest": True
    }
).save()

In [ ]:
portfolio

## Modifying Objects

Certain API objects such as Layers and Portfolios are mutable and can be modified after their initial creation. When these objects are modified they will retain their originally assigned ID. The Prime platform does not provide any built-in mechanisms for versioning or change history such that the original information stored with the object before modification is lost. However, there are client-side patterns that can be employed to store multiple versions of an object.

In the following example, we will update the layer we've created above with additional premium and reinstatement information. In addition, we will be creating a second layer which we will add to the previously created portfolio.

In [ ]:
# Retrieve originally created layer
layer = Layer.retrieve("[Enter layer GUID]")

In [ ]:
# Update layer attributes
layer.premium = MonetaryUnit(1e6, "USD")
layer.reinstatements = [ 
    Reinstatement(premium=0.5, brokerage=0.1), 
    Reinstatement(premium=0.2, brokerage=0.1) 
]
layer.save()

In [ ]:
# Create another layer
second_layer = Layer(
    # Type of layer: CatXL
    type="CatXL",
    # Loss sets associated with the layer
    loss_sets=[ls],
    # 3M USD attachment
    attachment=MonetaryUnit(13e6, "USD"),
    # 10M USD limit
    limit=MonetaryUnit(5e6, "USD"),
    # Franchise deductible is a required field, but we set it to 0 here.
    franchise=MonetaryUnit(0, "USD"),
    # The reinsurer's participation: 25%
    participation=0.10,
    # Inception date (optional, inclusive, supports `datetime`)
    inception_date="2022-01-01T00:00:00Z",
    # Expiry date (optional, exclusive, supports `datetime`)
    expiry_date="2023-01-01T00:00:00Z",
    # Description (optional)
    description="Test Layer 2 for Python Training",
    # Additional metadata fields (optional)
    meta_data={
        "Test-LayerId": 12346,
        "Test-Purpose": "Training",
        "Test-Version": 1,
        "Test-Latest": True,
    }
).save()

In [ ]:
# Add second layer to portfolio
portfolio = Portfolio.retrieve(portfolio.id)
portfolio.layers.append(second_layer)
portfolio.save()

## Creating Views

Views are one of the Prime platform's most important features. They are used to associate a financial structure, such as a Layer or Portfolio, and their respectively associated Loss Sets, with an Analysis Profile, to facilitate the calculation of metrics. Views are immutable objects that cannot be modified once they are created. Upon creation each view is assigned an ID which is distinct for data that encompasses the view. Two identical views, which produce identical metrics, are assigned identical IDs.

In order to create a Layer View we require a Layer object and an Analysis Profile. For this example, we will use the modified layer above `layer` and retrieve an appropriate Analysis Profile.

In [ ]:
from analyzere import LayerView

In [ ]:
lv = LayerView(
    analysis_profile=AnalysisProfile.retrieve("[Enter Analysis Profile GUID]"),
    layer=layer
).save()

In [ ]:
lv

Similarly, in order to create a Portfolio View of a portfolio, we require a portfolio object `portfolio` and an appropriate Analysis Profile.

In [ ]:
from analyzere import PortfolioView

In [ ]:
pv = PortfolioView(
    analysis_profile=AnalysisProfile.retrieve("[Enter Analysis Profile GUID]"),
    portfolio=portfolio,
    target_currency="USD",
).save()

In [ ]:
pv
# Note: Creating the Portfolio View has implicitly created Layer Views 
# for all of my layers in the portfolio. Also note that the Layer View ID 
# for my first layer is identical to the Layer View ID created above because 
# they refer to the same data.

It is also possible to change the default reporting currency for a Layer View or Portfolio View by specifying the `target_currency` attribute during construction.

In [ ]:
pv_gbp = PortfolioView(
    analysis_profile=AnalysisProfile.retrieve("[Enter Analysis Profile GUID]"),
    portfolio=portfolio,
    target_currency="GBP",
).save()

In [ ]:
pv_gbp

## Querying metrics

As indicated above, views are primarily used to obtain metrics for a specific Layer or Portfolio using the modelling configuration specified in the Analysis Profile. The Prime platform offers the ability to query a number of different risk metrics through a simple yet powerful interface. Some common risk metrics such as Expected Loss (AAL) have short-cut functions, but the unterlying calculations are identical.

In [ ]:
# Query various expected losses (AAL) from a Layer View
print("Expected loss (losses-only, no filters): ", lv.el())
print("Expected loss (losses-only, US Florida WS): ", lv.el(filter="US_Florida_WS"))
print(
    "Expected loss (net Prem/ReinPrem, no filters): ",
    lv.el(perspective="NetLoss,Premium,ReinstatementPremium")
)
print(
    "Expected loss (net ReinPrem, US Florida WS): ",
    lv.el(perspective="NetLoss,ReinstatementPremium", filter="US_Florida_WS")
)
print(
    "Expected loss OEP (losses-only, US Florida WS): ", 
    lv.el(filter="US_Florida_WS", aggregation_method="OEP")
)

In [ ]:
# Note that for stand-alone metrics participation is not applied by default.
print("Our Expected loss (losses-only, no filters): ", lv.el(apply_participation=True))
print(
    "Our Expected loss (losses-only, US Florida WS): ", 
    lv.el(apply_participation=True, filter="US_Florida_WS")
)
print(
    "Our Expected loss (net Prem/ReinPrem, no filters): ",
    lv.el(
        apply_participation=True,
        perspective="NetLoss,Premium,ReinstatementPremium"
    )
)
print(
    "Our Expected loss (net ReinPrem, US Florida WS): ",
    lv.el(
        perspective="NetLoss,ReinstatementPremium", 
        filter="US_Florida_WS",
        apply_participation=True
    )
)
print(
    "Expected loss OEP (losses-only, US Florida WS): ", 
    lv.el(
        filter="US_Florida_WS", 
        aggregation_method="OEP",
        apply_participation=True
    )
)

In [ ]:
# Similarly we can query the expected losses (AAL) for Portfolio Views
print("Expected loss (losses-only, no filters): ", pv.el())
print("Expected loss (losses-only, US Florida WS): ", pv.el(filter="US_Florida_WS"))
print(
    "Expected loss (net Prem/ReinPrem, no filters): ",
    pv.el(perspective="NetLoss,Premium,ReinstatementPremium")
)
print(
    "Expected loss (net ReinPrem, US Florida WS): ",
    pv.el(perspective="NetLoss,ReinstatementPremium", filter="US_Florida_WS")
)
print(
    "Expected loss OEP (losses-only, US Florida WS): ", 
    pv.el(
        filter="US_Florida_WS", 
        aggregation_method="OEP",
    )
)
# Note that for portfolio view metrics individual layer participations are always applied.

### VaR / TVaR / CTE

In [ ]:
# Tail Metrics are various statistics that are computed for the tail of the loss distribution
lv.tail_metrics(0.1)

In [ ]:
# Layer View tail metrics
return_period = 10
probability = 1.0 / return_period
print(
    f"Layer 1-in-{return_period} CTE (losses-only, no filter):",
    lv.tail_metrics(probability).mean
)
print(
    f"Layer 1-in-{return_period} CTE (losses-only, US Florida WS):",
    lv.tail_metrics(probability, filter="US_Florida_WS").mean
)
print(
    f"Layer 1-in-{return_period} CTE (net Prem/ReinPrem, no filter):",
    lv.tail_metrics(probability, perspective="NetLoss,Premium,ReinstatementPremium").mean
)
print(
    f"Layer 1-in-{return_period} CTE (net ReinPrem, US Florida WS):",
    lv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS"
    ).mean
)
print(
    f"Layer 1-in-{return_period} OEP CTE (net ReinPrem, US Florida WS):",
    lv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS",
        aggregation_method="OEP"
    ).mean
)

In [ ]:
# Portfolio View tail metrics (participation applied)
return_period = 10
probability = 1.0 / return_period
print(
    f"Portfolio 1-in-{return_period} CTE (losses-only, no filter):",
    pv.tail_metrics(probability).mean
)
print(
    f"Portfolio 1-in-{return_period} CTE (losses-only, US Florida WS):",
    pv.tail_metrics(probability, filter="US_Florida_WS").mean
)
print(
    f"Portfolio 1-in-{return_period} CTE (net Prem/ReinPrem, no filter):",
    pv.tail_metrics(probability, perspective="NetLoss,Premium,ReinstatementPremium").mean
)
print(
    f"Portfolio 1-in-{return_period} CTE (net ReinPrem, US Florida WS):",
    pv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS"
    ).mean
)
print(
    f"Portfolio 1-in-{return_period} OEP CTE (net ReinPrem, US Florida WS):",
    pv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS",
        aggregation_method="OEP"
    ).mean
)

### Exceedance Probability

The probability a loss reaches or exceeds a certain threshold. Used to determine attachment, exhaustion, and reinstatement probabilities.

In [ ]:
print(f"Layer attachment probability:", lv.ep(0.0).probability)
print(
    f"Layer occurrence exhaustion probability:", 
    lv.ep(
        lv.layer.limit.value, 
        inclusive_threshold=True,
        aggregation_method="OEP",
    ).probability
)
print(
    f"Layer aggregate exhaustion probability:", 
    lv.ep(
        lv.layer.limit.value*(len(lv.layer.reinstatements)+1), 
        inclusive_threshold=True
    ).probability
)
print(
    f"Layer reinstatement probability:", 
    lv.ep(
        lv.layer.limit.value, 
        inclusive_threshold=False,
    ).probability
)

## Performance Optimizations

HTTP REST requests incur some overhead through latency between the client and server, and processing overhead on the server side. Overall application throughput can be greatly enhanced by employing two techniques:

1. Vectorization: to reduce that overhead
2. Parallelism: to amortize the overhead

### Vectorization

Most metrics request offer the opportunity to specify a vector of probabilities or thresholds which bundle multiple requests into a single request.

In [ ]:
[ point.mean for point in lv.tail_metrics([1.0, 0.5, 0.2, 0.1]) ]

In [ ]:
[ 
    threshold.probability 
    for threshold in lv.ep([0.5e6, 1e6, 2e5, 3e6], inclusive_threshold=True) 
]

### Parallelism

Some parameters to metrics requests cannot be vectorized. For example, requesting metrics for different filters are always separate request. However, sometimes metrics for a particularly large number of filters and/or perspectives must be retrieved. In those situations it is beneficial to issue multiple requests to the platform concurrenly to amortize each individual request's overhead across parallel requests. For example:

In [ ]:
filters = [
    "Caribbean_WS",
    "Central_America_WS",
    "Mexico_WS",
    "US_Florida_WS",
    "US_Gulf_WS",
    "US_Mid_Atlantic_WS",
    "US_North_East_WS",
    "US_Texas_WS"
]
aggregation_methods = ["AEP", "OEP"]
perspectives = ["NetLoss", "NetLoss,ReinstatementPremium"]

In [ ]:
from itertools import product
requests = list(product(filters, aggregation_methods, perspectives))
print(f"Number of requests required: {len(requests)}")

In [ ]:
# Execute multiple requests in parallel using a ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor as Pool

with Pool(8) as pool:
    
    # Helper function
    def get_cte(filter, aggregation_method, perspective):
        return lv.tail_metrics(
            0.1, 
            filter=filter, 
            aggregation_method=aggregation_method, 
            perspective=perspective
        ).mean
    
    results = list(pool.map(lambda job: get_cte(*job), requests))

list(zip(requests, results))

## Advanced Metrics

### Co-Metrics

In [ ]:
# Contribution of Layer 2 to the Portfolio 1-in-10
pv.co_metrics(
    0.1, 
    component_id=pv.layer_views[1].id, 
    component_type="LayerView", 
    include_primary_metrics=True
)

In [ ]:
# Contribution of Layer 2 US_Florida_WS to the Portfolio 1-in-10
pv.co_metrics(
    0.1, 
    component_id=pv.layer_views[1].id, 
    component_type="LayerView",
    component_filter="US_Florida_WS",
    include_primary_metrics=True
).component_metrics.mean

### Window Metrics

In [ ]:
# 1-in-5 good vs. 1-in-5 bad years
pv.window_metrics([(0.8, 1.0), (0.0, 0.2)])